In [35]:
### CORRERLO INICIALENTE ###
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import sys
from pathlib import Path

import pandas as pd
import polars as pl
import dagster as dg
from kdags.resources.dplyr import *
from kdags.resources.tidyr import *
from io import BytesIO
import pdfplumber

from datetime import datetime
import re
import os
from kdags.definitions import *
from kdags.readr import *
from datetime import timedelta

context = dg.build_asset_context()
### IGNORAR ###

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
# Forma básica de leer un archivo Excel
df = pl.read_excel(
    Path(os.environ["ONEDRIVE_LOCAL_PATH"]) / "INSTRUCTIVOS/listado_instructivos.xlsx",
    infer_schema_length=0,
)
df = df.with_columns(
    pl.col("PROXIMA_REVISION_DATE").str.strptime(pl.Date, "%d-%m-%Y"),
)
correspondencia_instructivos = {
    "IG-0001-KCH (vencido)": "Instructivo de Bloqueo y Desenergización Camiones EléctricosIG-0001-KCH-REF-E-0001-KCH",
    "IG-0005-KCH (al día)": "Trabajo en altura físicaIG-0005-KCH-REF-E-0002-KCH",
    "IG-0014-KCH (vencido)": "Ingreso y salida de equipos de taller losa de lavado o similaresIG-0014-KCH-REF-E-0008-KCH",
    "KOP-I-1062-Mel (necesita verificación para actualización)": "Mantención preventiva y general de equipos CAEX 830E/930E/960EKOP-I-1062-MEL",
    "Cambio módulo de potencia (al día)": "Cambio de conjunto modulo de potenciaIT1-073-0315-FC-A000",
    "Cambio de acumulador de freno (vencido)": "Cambio de acumulador de frenoIT1-045-0814-FC-2G40",
    "Cambio de parrillas de forma individual (vencido)": "Cambio de parrillas de forma individualIT1-176-0321-HA-0960",
    "Cambio de rótulas de cilindro y/o barra de dirección (al día)": "Cambio rótula de cilindro y/o barra de direcciónIT1-118-0917-HA-4A00",
    "Cambio de freno de estacionamiento (al día)": "Cambio de freno de estacionamientoIT4-038-1214-HA-2F00",
    "Cambio ducto escape y silenciador (vencido)": "Cambio de ducto de escape y silenciadorIT1-177-0321-VO-0130",
    "Cambio de blower de parrilla (al día)": "Cambio de Blower de PariillasIT1-107-0916-PI-0967",
    "Cambio CMSD (al día)": "Cambio conjunto masa suspensión delanteraIT1-067-0115-FC-5A30",
    "Cambio de acumulador de dirección (vencido)": "Cambio acumulador de direcciónIT1-053-1014-FC-4AA0",
    "Cambio cilindro dirección (vencido)": "Cambio de cilindro de direcciónIT1-008-0214-FC-H1X0",
    "KOP-I-003–MEL": "Ingreso y salida de camiones DE EXTRACCIÓN 930E / 960E a naves de talleres 797 y 793 Minera EscondidaKOP-I-003-MEL",
}
correspondencia_invertida = {
    valor: clave for clave, valor in correspondencia_instructivos.items()
}

df = df.with_columns(
    (df["ACTIVIDAD"] + df["CODIGO_INSTRUCTIVO"])
    .replace(correspondencia_invertida, default=None)
    .alias("OSVALDO")
)
df = df.with_columns(
    pl.when(pl.col("PROXIMA_REVISION_DATE") <= pl.lit(datetime.now()))
    .then(pl.lit("vencido"))
    .when(
        pl.col("PROXIMA_REVISION_DATE") <= pl.lit(datetime.now() + timedelta(days=30))
    )
    .then(pl.lit("por_vencer"))
    .otherwise(pl.lit("actualizado"))
    .alias("estado_revision")
)
df.write_excel(
    Path(os.environ["ONEDRIVE_LOCAL_PATH"]) / "INSTRUCTIVOS/LISTADO_INSTRUCTIVOS.xlsx",
)

In [38]:
Path(os.environ["ONEDRIVE_LOCAL_PATH"]) / "INSTRUCTIVOS/LISTADO_INSTRUCTIVOS.xlsx"

WindowsPath('C:/Users/vales/OneDrive - Komatsu Ltd/INSTRUCTIVOS/LISTADO_INSTRUCTIVOS.xlsx')

In [27]:
(df["ACTIVIDAD"] + df["CODIGO_INSTRUCTIVO"]).to_list()

['Instructivo de Bloqueo y Desenergización Camiones EléctricosIG-0001-KCH-REF-E-0001-KCH',
 'Instructivo de subida y bajada de equiposIG-0006-KCH-REF-E-0002-KCH',
 'Trabajo en altura físicaIG-0005-KCH-REF-E-0002-KCH',
 'Trabajo con carga suspendidaIG-0007-KCH-REF-E-0003-KCH',
 'Ingreso y salida de equipos de taller losa de lavado o similaresIG-0014-KCH-REF-E-0008-KCH',
 'Instructivo uso y cuidado de equipos de protección personalIG-0016-KCH-REF-E-0010-KCH',
 'Ejecución MP equipos KomatsuIG-0020-KCH-REF-E-009-KCH',
 'Cambio de sellos de puertas de cabina de operadorIT4-067-0615-HA-5520',
 'Cambio asiento operadorIT4-073-1117-PL-5700',
 'Cambio mangueras hidráulicasIT4-072-1117-PL-6A02',
 'Carga de nitrógeno y llenado de aceite suspensión delanteraIT1-005-0114-FC-CN2',
 'Cambio de cilindro de direcciónIT1-008-0214-FC-H1X0',
 'Cambio de conjunto motor de tracciónIT1-007-0214-FC-0970',
 'Cambio suspensión traseraIT1-013-0414-FC-5A4',
 'Carga de nitrógeno y llenado de aceite suspensión tras

In [24]:
import polars as pl

# Forma básica de leer un archivo Excel
df = pl.read_excel(
    Path(os.environ["ONEDRIVE_LOCAL_PATH"]) / "INSTRUCTIVOS/listado_instructivos.xlsx",
    infer_schema_length=0,
)
df = df.with_columns(
    pl.coalesce(
        pl.col("PROXIMA_REVISION_DATE").str.strptime(pl.Date, "%d-%m-%Y", strict=False),
        pl.col("PROXIMA_REVISION_DATE").str.strptime(pl.Date, "%d/%m/%Y", strict=False),
    ).alias("PROXIMA_REVISION_DATE")
)
df = df.with_columns(
    pl.when(pl.col("PROXIMA_REVISION_DATE") < pl.lit(datetime.now()))
    .then(pl.lit("vencido"))
    .when(pl.col("PROXIMA_REVISION_DATE") > pl.lit(datetime.now()))
    .then(pl.lit("por_vencer"))
    .otherwise(pl.lit("actualizado"))
    .alias("estado_revision")
).with_columns(
    file_url=pl.lit(
        "https://globalkomatsu.sharepoint.com/sites/KCHCLSP00022/Shared%20Documents/01.%20%C3%81REAS%20KCH/1.6%20CONFIABILIDAD/CAEX/SERVICIO_AL_CLIENTE/INSTRUCTIVOS/DOCUMENTOS/"
    )
    + pl.col("TIPO_INSTRUCTIVO")
    + pl.lit("/")
    + pl.col("TIPO_DOCUMENTO")
    + pl.lit("/")
    + pl.col("CODIGO_INSTRUCTIVO")
    + pl.lit(".pdf")
)

df.write_excel(
    Path(os.environ["ONEDRIVE_LOCAL_PATH"]) / "INSTRUCTIVOS/LISTADO_INSTRUCTIVOS.xlsx",
)

df

ACTIVIDAD,TIPO_INSTRUCTIVO,TIPO_DOCUMENTO,CODIGO_INSTRUCTIVO,Actualizado,FIRMA,Ultima Actualizacion,PROXIMA_REVISION_DATE,Comentario,estado_revision,file_url
str,str,str,str,str,str,str,date,str,str,str
"""Instructivo de Bloqueo y Desen…","""MANTENIMIENTO""","""INSTRUCTIVO_TECNICO""","""IG-0001-KCH-REF-E-0001-KCH""","""Actualizado""","""Electronica""","""26-08-2024""",2026-08-26,null,"""por_vencer""","""https://globalkomatsu.sharepoi…"
"""Instructivo de subida y bajada…","""MANTENIMIENTO""","""INSTRUCTIVO_TECNICO""","""IG-0006-KCH-REF-E-0002-KCH""","""Actualizado""","""Electronica""","""15-11-2023""",2025-11-15,null,"""por_vencer""","""https://globalkomatsu.sharepoi…"
"""Trabajo en altura física""","""MANTENIMIENTO""","""INSTRUCTIVO_TECNICO""","""IG-0005-KCH-REF-E-0002-KCH""","""Actualizado""","""Electronica""","""2024-01-06 00:00:00""",null,null,"""actualizado""","""https://globalkomatsu.sharepoi…"
"""Trabajo con carga suspendida""","""MANTENIMIENTO""","""INSTRUCTIVO_TECNICO""","""IG-0007-KCH-REF-E-0003-KCH""","""Actualizado""","""Electronica""","""15-11-203""",2025-11-14,null,"""por_vencer""","""https://globalkomatsu.sharepoi…"
"""Ingreso y salida de equipos de…","""MANTENIMIENTO""","""INSTRUCTIVO_TECNICO""","""IG-0014-KCH-REF-E-0008-KCH""","""Actualizado""","""Electronica""","""2024-05-02 00:00:00""",null,"""Error en fecha de proxima revi…","""actualizado""","""https://globalkomatsu.sharepoi…"
…,…,…,…,…,…,…,…,…,…,…
"""Cambio de Estanque hidraulico""","""MANTENIMIENTO""","""INSTRUCTIVO_TECNICO""","""IT1-112-1116-HA-6010""","""Actualizado""","""Electronica""","""28-11-2023""",2025-11-27,null,"""por_vencer""","""https://globalkomatsu.sharepoi…"
"""Ingreso y salida de camiones D…","""MANTENIMIENTO""","""INTRUCTIVO_TRABAJO""","""KOP-I-003-MEL""","""Actualizado""","""Electro/Manual""","""2024-09-01 00:00:00""",null,null,"""actualizado""","""https://globalkomatsu.sharepoi…"
"""Inspección de puntos calientes…","""MANTENIMIENTO""","""INTRUCTIVO_TRABAJO""","""KOP-I-1251-MEL""","""Actualizado""","""Electro/Manual""","""28-05-2024""",2026-05-28,null,"""por_vencer""","""https://globalkomatsu.sharepoi…"


In [3]:
# codigo para buscar archivos en una carpeta.
dir_path = Path(os.environ["ONEDRIVE_LOCAL_PATH"]) / "INSTRUCTIVOS_TECNICOS/DOCUMENTOS"
files = [str(f) for f in Path(dir_path).rglob("*") if f.is_file()]
files_df = pl.DataFrame({"filepath": files})
files_df
import polars as pl

# Extraer el nombre del archivo usando expresiones regulares
files_df = files_df.with_columns(
    pl.col("filepath").str.extract(r"([^\\\/]+)\.pdf$", 1).alias("filename")
)
files_df

filepath,filename
str,str
"""C:\Users\vales\OneDrive - Koma…","""IG-0001-KCH-REF-E-0001-KCH"""
"""C:\Users\vales\OneDrive - Koma…","""IG-0005-KCH-REF-E-0002-KCH"""
"""C:\Users\vales\OneDrive - Koma…","""IG-0006-KCH-REF-E-0002-KCH"""
"""C:\Users\vales\OneDrive - Koma…","""IG-0007-KCH-REF-E-0003-KCH"""
"""C:\Users\vales\OneDrive - Koma…","""IG-0014-KCH-REF-E-0008-KCH"""
…,…
"""C:\Users\vales\OneDrive - Koma…","""IT6-021-0423-RS-K@24"""
"""C:\Users\vales\OneDrive - Koma…","""KOP-I-003-MEL"""
"""C:\Users\vales\OneDrive - Koma…","""KOP-I-1062-MEL"""


In [4]:
# Join básico entre df y files_df usando la columna filename
mezcla_df = (
    df.join(
        files_df,
        on="filename",  # Columna común para unir
        how="outer",  # Tipo de join (inner por defecto)
    ).with_columns(
        new_filepath=pl.lit(os.environ["ONEDRIVE_LOCAL_PATH"])
        + pl.lit("\\INSTRUCTIVOS_TECNICOS\\DOCUMENTOS\\")
        + pl.col("TIPO_INSTRUCTIVO")
        + pl.lit("\\")
        + pl.col("TIPO_DOCUMENTO")
        + pl.lit("\\")
        + pl.col("filename")
        + pl.lit(".pdf")
    )
    # .filter((pl.col("filepath").is_not_null()) & (pl.col("new_filepath").is_not_null()))
    # .drop_nulls(subset=["filepath", "new_filepath"])
)
mezcla_df.filter(mezcla_df["filename_right"].is_null())  # ["filename"].to_list()

item,DOCUMENTO,TIPO_INSTRUCTIVO,TIPO_DOCUMENTO,filename,AT,Actualizado,Firma,Ultima Actualizacion,Proxima Revision,Comentario,IT (ubicación),filepath,filename_right,new_filepath
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str


""Necesito hacer un código utilizando un pandas dataframe y que vaya iterando através através de cada fila. Acá que tome la columna llamada filepath, y que mediante pathlib y creando las carpeta necesarias copie los archivos en la columna filepath y los genere en los valores de la nueva columna llamda new_filepath en base a un dataframe llamada pandas_df

In [5]:
import pandas as pd
from pathlib import Path
import shutil
import os

# Suponiendo que ya tienes un DataFrame llamado pandas_df con columnas 'filepath' y 'new_filepath'


def copy_files_from_dataframe(df):
    """
    Copia archivos desde las rutas en la columna 'filepath' a las rutas en 'new_filepath',
    creando los directorios necesarios en el proceso.

    Args:
        df (pandas.DataFrame): DataFrame con las columnas 'filepath' y 'new_filepath'

    Returns:
        dict: Estadísticas de la operación (archivos copiados, errores, etc.)
    """
    stats = {
        "total_files": len(df),
        "successful_copies": 0,
        "failed_copies": 0,
        "errors": [],
    }

    # Iterar por cada fila del DataFrame
    for index, row in df.iterrows():
        source_path = Path(row["filepath"])
        target_path = Path(row["new_filepath"])

        try:
            # Verificar si el archivo de origen existe
            if not source_path.exists():
                raise FileNotFoundError(
                    f"Archivo de origen no encontrado: {source_path}"
                )

            # Crear directorios de destino si no existen
            target_dir = target_path.parent
            target_dir.mkdir(parents=True, exist_ok=True)

            # Copiar el archivo
            shutil.copy2(source_path, target_path)

            # Actualizar estadísticas
            stats["successful_copies"] += 1

            # Imprimir progreso cada 10 archivos
            if stats["successful_copies"] % 10 == 0:
                print(
                    f"Progreso: {stats['successful_copies']}/{stats['total_files']} archivos copiados"
                )

        except Exception as e:
            # Registrar el error
            error_info = {
                "index": index,
                "source": str(source_path),
                "target": str(target_path),
                "error": str(e),
            }
            stats["errors"].append(error_info)
            stats["failed_copies"] += 1
            print(f"Error al copiar archivo {index}: {e}")

    # Imprimir resumen
    print(f"\nResumen:")
    print(f"Total de archivos: {stats['total_files']}")
    print(f"Archivos copiados exitosamente: {stats['successful_copies']}")
    print(f"Archivos con errores: {stats['failed_copies']}")

    # Opcional: guardar los errores en un archivo CSV para revisión
    if stats["failed_copies"] > 0:
        error_df = pd.DataFrame(stats["errors"])
        error_file = "errores_copia_archivos.csv"
        error_df.to_csv(error_file, index=False)
        print(f"Detalles de errores guardados en: {error_file}")

    return stats


# copy_files_from_dataframe(mezcla_df.dropna(subset=["filepath", "new_filepath"]))
copy_files_from_dataframe(mezcla_df.to_pandas())

Progreso: 10/93 archivos copiados
Progreso: 20/93 archivos copiados
Progreso: 30/93 archivos copiados
Progreso: 40/93 archivos copiados
Progreso: 50/93 archivos copiados
Progreso: 60/93 archivos copiados
Progreso: 70/93 archivos copiados
Progreso: 80/93 archivos copiados
Progreso: 90/93 archivos copiados

Resumen:
Total de archivos: 93
Archivos copiados exitosamente: 93
Archivos con errores: 0


{'total_files': 93, 'successful_copies': 93, 'failed_copies': 0, 'errors': []}

In [10]:
df.columns

['item ',
 'DOCUMENTO',
 'TIPO_INSTRUCTIVO',
 'TIPO_DOCUMENTO',
 'filename',
 'AT',
 'Actualizado',
 'Firma',
 'Ultima Actualizacion',
 'Proxima Revision',
 'Comentario',
 'IT (ubicación)']

In [11]:
df.drop(["item "])

DOCUMENTO,TIPO_INSTRUCTIVO,TIPO_DOCUMENTO,filename,AT,Actualizado,Firma,Ultima Actualizacion,Proxima Revision,Comentario,IT (ubicación)
str,str,str,str,str,str,str,str,str,str,str
"""Instructivo de Bloqueo y Desen…","""MANTENIMIENTO""","""INSTRUCTIVO_TECNICO""","""IG-0001-KCH-REF-E-0001-KCH""","""S.Cepeda""","""Actualizado""","""Electronica""","""26-08-2024""","""26-08-2026""",null,"""https://globalkomatsu.sharepoi…"
"""Instructivo de subida y bajada…","""MANTENIMIENTO""","""INSTRUCTIVO_TECNICO""","""IG-0006-KCH-REF-E-0002-KCH""","""S.Cepeda""","""Actualizado""","""Electronica""","""15-11-2023""","""15-11-2025""",null,"""https://globalkomatsu.sharepoi…"
"""Trabajo en altura física""","""MANTENIMIENTO""","""INSTRUCTIVO_TECNICO""","""IG-0005-KCH-REF-E-0002-KCH""","""S.Cepeda""","""Actualizado""","""Electronica""","""2024-01-06 00:00:00""","""2026-01-06 00:00:00""",null,"""https://globalkomatsu.sharepoi…"
"""Trabajo con carga suspendida""","""MANTENIMIENTO""","""INSTRUCTIVO_TECNICO""","""IG-0007-KCH-REF-E-0003-KCH""","""S.Cepeda""","""Actualizado""","""Electronica""","""15-11-203""","""14-11-2025""",null,"""https://globalkomatsu.sharepoi…"
"""Ingreso y salida de equipos de…","""MANTENIMIENTO""","""INSTRUCTIVO_TECNICO""","""IG-0014-KCH-REF-E-0008-KCH""","""S.Cepeda""","""Actualizado""","""Electronica""","""2024-05-02 00:00:00""","""2026-05-02 00:00:00""","""Error en fecha de proxima revi…","""https://globalkomatsu.sharepoi…"
…,…,…,…,…,…,…,…,…,…,…
"""Cambio de Estanque hidraulico""","""MANTENIMIENTO""","""INSTRUCTIVO_TECNICO""","""IT1-112-1116-HA-6010""","""S.Cepeda""","""Actualizado""","""Electronica""","""28-11-2023""","""27-11-2025""",null,null
"""Ingreso y salida de camiones D…","""MANTENIMIENTO""","""INTRUCTIVO_TRABAJO""","""KOP-I-003-MEL""",null,"""Actualizado""","""Electro/Manual""","""2024-09-01 00:00:00""","""2026-09-01 00:00:00""",null,"""DRS MEL - KOP-I-003-MEL.pdf - …"
"""Inspección de puntos calientes…","""MANTENIMIENTO""","""INTRUCTIVO_TRABAJO""","""KOP-I-1251-MEL""",null,"""Actualizado""","""Electro/Manual""","""28-05-2024""","""28-05-2026""",null,"""DRS MEL - KOP-I-1251-MEL.pdf -…"


In [6]:
mezcla_df

item,DOCUMENTO,TIPO_INSTRUCTIVO,TIPO_DOCUMENTO,filename,AT,Actualizado,Firma,Ultima Actualizacion,Proxima Revision,Comentario,IT (ubicación),filepath,filename_right,new_filepath
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""1""","""Instructivo de Bloqueo y Desen…","""MANTENIMIENTO""","""INSTRUCTIVO_TECNICO""","""IG-0001-KCH-REF-E-0001-KCH""","""S.Cepeda""","""Actualizado""","""Electronica""","""26-08-2024""","""26-08-2026""",null,"""https://globalkomatsu.sharepoi…","""C:\Users\vales\OneDrive - Koma…","""IG-0001-KCH-REF-E-0001-KCH""","""C:\Users\vales\OneDrive - Koma…"
"""3""","""Trabajo en altura física""","""MANTENIMIENTO""","""INSTRUCTIVO_TECNICO""","""IG-0005-KCH-REF-E-0002-KCH""","""S.Cepeda""","""Actualizado""","""Electronica""","""2024-01-06 00:00:00""","""2026-01-06 00:00:00""",null,"""https://globalkomatsu.sharepoi…","""C:\Users\vales\OneDrive - Koma…","""IG-0005-KCH-REF-E-0002-KCH""","""C:\Users\vales\OneDrive - Koma…"
"""2""","""Instructivo de subida y bajada…","""MANTENIMIENTO""","""INSTRUCTIVO_TECNICO""","""IG-0006-KCH-REF-E-0002-KCH""","""S.Cepeda""","""Actualizado""","""Electronica""","""15-11-2023""","""15-11-2025""",null,"""https://globalkomatsu.sharepoi…","""C:\Users\vales\OneDrive - Koma…","""IG-0006-KCH-REF-E-0002-KCH""","""C:\Users\vales\OneDrive - Koma…"
"""4""","""Trabajo con carga suspendida""","""MANTENIMIENTO""","""INSTRUCTIVO_TECNICO""","""IG-0007-KCH-REF-E-0003-KCH""","""S.Cepeda""","""Actualizado""","""Electronica""","""15-11-203""","""14-11-2025""",null,"""https://globalkomatsu.sharepoi…","""C:\Users\vales\OneDrive - Koma…","""IG-0007-KCH-REF-E-0003-KCH""","""C:\Users\vales\OneDrive - Koma…"
"""5""","""Ingreso y salida de equipos de…","""MANTENIMIENTO""","""INSTRUCTIVO_TECNICO""","""IG-0014-KCH-REF-E-0008-KCH""","""S.Cepeda""","""Actualizado""","""Electronica""","""2024-05-02 00:00:00""","""2026-05-02 00:00:00""","""Error en fecha de proxima revi…","""https://globalkomatsu.sharepoi…","""C:\Users\vales\OneDrive - Koma…","""IG-0014-KCH-REF-E-0008-KCH""","""C:\Users\vales\OneDrive - Koma…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""78""","""Instalación de ductos de admis…","""MANTENIMIENTO""","""INSTRUCTIVO_TECNICO""","""IT6-021-0423-RS-K@24""","""A.Moraleda""","""Por vencer""","""Electronica""","""15/04/2023""","""14/04/2025""","""Falta firma de analista docume…","""DRS MEL - IT6-021-0423-RS-K@24…","""C:\Users\vales\OneDrive - Koma…","""IT6-021-0423-RS-K@24""","""C:\Users\vales\OneDrive - Koma…"
"""90""","""Ingreso y salida de camiones D…","""MANTENIMIENTO""","""INTRUCTIVO_TRABAJO""","""KOP-I-003-MEL""",null,"""Actualizado""","""Electro/Manual""","""2024-09-01 00:00:00""","""2026-09-01 00:00:00""",null,"""DRS MEL - KOP-I-003-MEL.pdf - …","""C:\Users\vales\OneDrive - Koma…","""KOP-I-003-MEL""","""C:\Users\vales\OneDrive - Koma…"
"""93""","""Mantención preventiva y genera…","""MANTENIMIENTO""","""INTRUCTIVO_TRABAJO""","""KOP-I-1062-MEL""",null,"""Actualizado""","""Electro/Manual""","""2023-01-06 00:00:00""","""2025-01-06 00:00:00""",null,"""https://globalkomatsu.sharepoi…","""C:\Users\vales\OneDrive - Koma…","""KOP-I-1062-MEL""","""C:\Users\vales\OneDrive - Koma…"


In [13]:
mezcla_df["new_filepath"][0]

'C:\\Users\\vales\\OneDrive - Komatsu Ltd\\INSTRUCTIVOS_TECNICOS\\DOCUMENTOS\\MANTENIMIENTO\\INSTRUCTIVO_TECNICO\\IG-0001-KCH-REF-E-0001-KCH.pdf'